In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [7]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [8]:
data.shape

(50000, 2)

In [9]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [15]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [18]:
train_data,test_data = train_test_split(data,test_size=0.2,random_state=42)

In [19]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [20]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [21]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [22]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [23]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [24]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [38]:
model = Sequential()
model.add(Embedding(input_dim = 5000,output_dim = 128,input_length = 200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))

In [39]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [41]:
model.fit(X_train,Y_train,epochs=5,batch_size =2000,validation_split = 0.2)

Epoch 1/5
16/16 [==============================] - 231s 14s/step - loss: 0.6734 - accuracy: 0.6082 - val_loss: 0.6503 - val_accuracy: 0.5805
Epoch 2/5
16/16 [==============================] - 238s 15s/step - loss: 0.5972 - accuracy: 0.6796 - val_loss: 0.5423 - val_accuracy: 0.7749
Epoch 3/5
16/16 [==============================] - 238s 15s/step - loss: 0.4258 - accuracy: 0.8203 - val_loss: 0.3562 - val_accuracy: 0.8451
Epoch 4/5
16/16 [==============================] - 232s 15s/step - loss: 0.3091 - accuracy: 0.8727 - val_loss: 0.3099 - val_accuracy: 0.8729
Epoch 5/5
16/16 [==============================] - 228s 14s/step - loss: 0.2576 - accuracy: 0.8996 - val_loss: 0.2963 - val_accuracy: 0.8777


In [42]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 28s 87ms/step - loss: 0.2879 - accuracy: 0.8834
Test Loss: 0.2878814935684204
Test Accuracy: 0.883400022983551


In [43]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [45]:
# example usage
new_review = "time wasting movies."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 54ms/step
The sentiment of the review is: negative


In [46]:
# example usage
new_review = "This movie was fantastic"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 92ms/step
The sentiment of the review is: positive
